In [ ]:
!apt update && !apt upgrade
!apt install git wget libeigen3-dev -y

In [ ]:
!git clone https://github.com/guochengqian/Magic123.git

In [ ]:
%cd Magic123

In [ ]:
%pip install -r requirements.txt

In [ ]:
%cd pretrained/zero123
!wget https://huggingface.co/cvlab/zero123-weights/resolve/main/105000.ckpt
%cd ../../

In [ ]:
!mkdir -p pretrained/midas
%cd pretrained/midas
!wget https://github.com/isl-org/MiDaS/releases/download/v3_1/dpt_beit_large_512.pt
%cd ../../

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="scorp_side"
image_path = f'/notebooks/threestudio/load/images/{workspace}_rgba.png'
image_path_without_rgba = image_path.replace('_rgba', '')
save_path = f'./output/{workspace}'
prompt = '''
'''
# If the image contains non-front-facing objects, specifying the approximate elevation 
# and azimuth angle by setting data.default_elevation_deg and data.default_azimuth_deg can be helpful. 
# In threestudio, top is elevation +90 and bottom is elevation -90; left is azimuth -90 and right is azimuth +90.
default_elevation_deg = 0
default_azimuth_deg = 0
GPU_IDX = 0
init_token='scorpside'
token_name = f'_{init_token}_'

In [ ]:
%run preprocess_image.py --path {image_path_without_rgba}

In [ ]:
import os

MODEL_NAME = "runwayml/stable-diffusion-v1-5"
DATA_DIR =image_path
OUTPUT_DIR = save_path
placeholder_token=token_name
cmd = f"""
textual-inversion/textual_inversion.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir={DATA_DIR} \
  --learnable_property="object" \
  --placeholder_token={placeholder_token} \
  --initializer_token={init_token} \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=3000 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir={OUTPUT_DIR} \
  --use_augmentations
  
"""
%run {cmd}
  
%run guidance/sd_utils.py \
  --text="A high-resolution DSLR image of <token>" \
  --learned_embeds_path={OUTPUT_DIR} \
  --workspace={OUTPUT_DIR}


In [ ]:
import os

RUN_ID = "jobname_for_the_first_stage"
RUN_ID2 = "jobname_for_the_second_stage"
DATA_DIR, IMAGE_NAME = os.path.split(image_path)
step1 = True # change this according to your needs
step2 = True # change this according to your needs
if step1:
    !CUDA_VISIBLE_DEVICES=0 python main.py -O \
        --text "A high-resolution DSLR image of <token>" \
        --sd_version 1.5 \
        --image {image_path} \
        --learned_embeds_path {OUTPUT_DIR}/learned_embeds.bin \
        --workspace out/magic123-{RUN_ID}-coarse/$dataset/magic123_${FILENAME}_${RUN_ID}_coarse \
        --optim adam \
        --iters 5000 \
        --guidance SD zero123 \
        --lambda_guidance 1.0 40 \
        --guidance_scale 100 5 \
        --latent_iter_ratio 0 \
        --normal_iter_ratio 0.2 \
        --t_range 0.2 0.6 \
        --bg_radius -1 \
        --save_mesh 

if step2:
    !CUDA_VISIBLE_DEVICES=0 python main.py -O \
        --text "A high-resolution DSLR image of <token>" \
        --sd_version 1.5 \
        --image {image_path} \
        --learned_embeds_path {OUTPUT_DIR}/learned_embeds.bin  \
        --workspace out/magic123-{RUN_ID}-{RUN_ID2}/$dataset/magic123_${FILENAME}_${RUN_ID}_${RUN_ID2} \
        --dmtet --init_ckpt out/magic123-{RUN_ID}-coarse/$dataset/magic123_${FILENAME}_${RUN_ID}_coarse/checkpoints/magic123_${FILENAME}_${RUN_ID}_coarse.pth \
        --iters 5000 \
        --optim adam \
        --latent_iter_ratio 0 \
        --guidance SD zero123 \
        --lambda_guidance 1e-3 0.01 \
        --guidance_scale 100 5 \
        --rm_edge \
        --bg_radius -1 \
        --save_mesh 